In [52]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [53]:
from dataclasses import dataclass

@dataclass
class MetOceanParameters:
    Hm0: float        # Significant wave height [m]
    Tp: float         # Peak wave period [s]
    T02: float        # Mean zero-crossing period [s]
    CS_total: float   # Depth-averaged current speed [m/s]
    WS160: float      # Wind speed at 160 m [m/s]
    WS160_adj: float  # Wind speed at 160 m (+7%) [m/s]
    WL_tot: float     # Total water level [m MSL]
    Hmax: float       # Maximum wave height [m]
    T_Hmax: float     # Wave period associated with Hmax [s]
    Cmax_SWL: float   # Max crest level w.r.t. SWL [m]
    Cmax_MSL: float   # Max crest level w.r.t. MSL [m]
    WL_total: float      # Total water level (HWL_tot or LWL_tot) [mMSL]
    WL_residual: float   # Residual water level (HWL_res or LWL_res) [m]

# -------------------------
# 200-year return period
# -------------------------

HWL = MetOceanParameters(
    Hm0=9.9,
    Tp=15.3,
    T02=9.8,
    CS_total=0.5,
    WS160=29.7,
    WS160_adj=31.8,
    WL_tot=1.3,
    Hmax=19.2,
    T_Hmax=12.1,
    Cmax_SWL=12.8,
    Cmax_MSL=14.6,
    WL_total=2.4,
    WL_residual=2.2
)

LWL = MetOceanParameters(
    Hm0=8.7,
    Tp=13.4,
    T02=8.5,
    CS_total=0.5,
    WS160=32.9,
    WS160_adj=35.2,
    WL_tot=-0.2,
    Hmax=16.6,
    T_Hmax=10.5,
    Cmax_SWL=11.0,
    Cmax_MSL=10.7,
    WL_total=-1.7,
    WL_residual=-1.4
)


## Parameters

In [54]:
# General constants
g = 9.81 # m/s^2
rho_w = 1025 # kg/m^3
rho_s = 2650 # kg/m^3
nu = 1.33e-6 # m^2/s
bodemdiepte = 40 # mMSL
Kv = 1.4 # Velocity coefficient
Delta_v = (rho_s - rho_w) / rho_w # Dimensionless

## Armour Layer
### Boek methode

Diameter van de armour layer wordt hieronder bepaald via de methode boek: de adpated shield approach. Alleen snelheid (u_c) en waterdiepte (h) zullen varieren tussen weeromstandigheden.
- De waarde van Kv moet nog geverifieerd worden. Of bij docenten of uit literatuur 

In [55]:
def calculate_d_n50(u_c, H, T, h, psi_c, k_r, K_s=1.0, K_v=Kv, delta=Delta_v):
    """
    Calculate the characteristic stone diameter d_n50 for an armour layer (with adapted Shields formula).

    Parameters 
    ----------
    u_c : float
        Current velocity depth-averaged [m/s]
    H : float
        Wave height [m]
    T : float
        Wave period [s]
    h : float
        Water depth [m]
    psi_c : float, optional
        Critical motion parameter [-]
    k_r : float
        bottom_roughness [-]
    K_s : float, optional
        Strength reduction factor for stones on a slope [-]
    K_v : float, optional
        Velocity/turbulence factor [-]
    delta : float, optional
        Relative density (Δ = (rho_s - rho_w) / rho_w) [-]

    Returns
    -------
    d_n50 : float
        Required median stone diameter [m]
    """

    C = 50.0  # initial Chezy coefficient guess
    omega = 2.0 * np.pi / T
    k = omega**2 / g  # Initial guess for wave number
    for _ in range(10):
        k = omega**2 / (g * np.tanh(k * h))  # Update k using dispersion relation

    a_b = (H / 2.0) / np.sinh(k * h)
    u_b_hat = omega * a_b
    c_f = 0.237 * (a_b / k_r)**(-0.52)
    
    u_star_r_mean = np.sqrt((g / C**2) * u_c**2 + (c_f / 2) * u_b_hat**2 * 0.5)
    d_n50 = (K_v**2 * u_star_r_mean**2) / (K_s * psi_c * delta * C**2)

    return u_star_r_mean, d_n50

In [56]:
res_HWL = calculate_d_n50(HWL.CS_total, HWL.Hm0, HWL.Tp, HWL.WL_total + bodemdiepte, 0.3, 0.1)
res_LWL = calculate_d_n50(LWL.CS_total, LWL.Hm0, LWL.Tp, LWL.WL_total + bodemdiepte, 0.3, 0.1)
print(f'For HWL conditions:  \n\t- u_c = {HWL.CS_total} m/s,\n\t- H = {HWL.Hm0} m,\n\t- T = {HWL.Tp} s,\n\t- h = {HWL.WL_total + bodemdiepte} m\nThe resulting time-averaged shear velocity:\t u_star_r_mean = {res_HWL[0]:.4f} m/s\n and the required median stone diameter d_n50is:\t d_n50 = {res_HWL[1]:.5f} m.\n')
print(f'For LWL conditions:  \n\t- u_c = {LWL.CS_total} m/s,\n\t- H = {LWL.Hm0} m,\n\t- T = {LWL.Tp} s,\n\t- h = {LWL.WL_total + bodemdiepte} m\nThe resulting time-averaged shear velocity:\t u_star_r_mean = {res_LWL[0]:.4f} m/s\n and the required median stone diameter d_n50is:\t d_n50 = {res_LWL[1]:.5f} m.')

For HWL conditions:  
	- u_c = 0.5 m/s,
	- H = 9.9 m,
	- T = 15.3 s,
	- h = 42.4 m
The resulting time-averaged shear velocity:	 u_star_r_mean = 0.1667 m/s
 and the required median stone diameter d_n50is:	 d_n50 = 0.00005 m.

For LWL conditions:  
	- u_c = 0.5 m/s,
	- H = 8.7 m,
	- T = 13.4 s,
	- h = 38.3 m
The resulting time-averaged shear velocity:	 u_star_r_mean = 0.1561 m/s
 and the required median stone diameter d_n50is:	 d_n50 = 0.00004 m.


### ARMOUR LAYER  HASPRO


In [153]:
d_50 = 0.25 # m [from LM_A   10-60 kg]
D_p = 9 # m
g = 9.81 # m/s^2
rho_w = 1025 # kg/m^3
rho_stone = 2800 # kg/m^3 [from stone grading table]
nu = 1.33e-6 # m^2/s
H_s = 9.9 # m
T_p = 10.5 # s
u_c = 0.5 # m/s
h_w = 40 # m [water depth as site]
D_n50 = 0.21 # m [from LM_A   10-60 kg]
n = 2.5

def calc_hydrodynamic_params(T_p, H_s, u_c, h_w, D_n50, D_p, g=9.81, ):
    T_z = T_p / 1.3 

    u_m_bed = np.sqrt(2) * (H_s / 4) * np.sqrt(g / h_w) * np.exp(-((3.65 / T_z) * np.sqrt(h_w / g))**2.1)

    L_w_peak_period = 200
    for repetition in range(10):
        L_w_peak_period = g * T_p**2 / (2 * np.pi) * np.tanh((2 * np.pi * h_w) / L_w_peak_period)
    
    K_top =  np.sinh(2 * np.pi * h_w / L_w_peak_period) / np.sinh(2 * np.pi * (h_w - 4 * D_n50) / L_w_peak_period)

    u_m_top = u_m_bed * K_top

    A_w_a = (u_m_top * T_p) / (2 * np.pi)

    KC_c = np.abs(u_c) * T_p / D_p 
    KC_w = u_m_bed * T_p / D_p 
    KC_total = KC_c + KC_w

    print(f'')

    return u_m_bed, u_m_top, A_w_a, KC_c, KC_w, KC_total , T_z, L_w_peak_period


u_m_bed, u_m_top, A_w_a, KC_c, KC_w, KC_total, T_z, L_w_peak_period = calc_hydrodynamic_params(T_p, H_s, u_c, h_w, D_n50, D_p)
print(f'u_m_bed = {u_m_bed:.4f} m/s\nu_m_top = {u_m_top:.4f} m/s\nA_w_a = {A_w_a:.4f} m\nKC_c = {KC_c:.4f}\nKC_w = {KC_w:.4f}\nKC_total = {KC_total:.4f}\nT_z = {T_z:.4f} s\nL_w_peak_period = {L_w_peak_period:.4f} m')



u_m_bed = 0.7595 m/s
u_m_top = 0.7877 m/s
A_w_a = 1.3163 m
KC_c = 0.5833
KC_w = 0.8861
KC_total = 1.4695
T_z = 8.0769 s
L_w_peak_period = 158.3219 m


In [154]:


def bed_shear_stresses(d_50, n, A_w_a, u_m_top, rho_w):
    k_s = n * d_50

    ampl_ratio = A_w_a / k_s

    wave_friction_coefficient = 0.237 * (ampl_ratio)**(-0.52)

    u_ww = np.sqrt(wave_friction_coefficient / 2) * u_m_top

    tau_bed_wave = rho_w * u_ww**2

    bed_roughness_length = k_s / 30

    drag_coefficient = (0.4 / (np.log(h_w / bed_roughness_length) - 1))**2

    shear_velocity_current = np.sqrt(drag_coefficient) * u_c

    tau_bed_current = rho_w * shear_velocity_current**2

    tau_mean = tau_bed_current + 1.2 * tau_bed_current * (tau_bed_wave / (tau_bed_current + tau_bed_wave))**3.2

    tau_max = tau_mean + tau_bed_wave 

    return wave_friction_coefficient, tau_bed_wave, tau_bed_current, tau_mean, tau_max 

wave_friction_coefficient, tau_bed_wave, tau_bed_current, tau_mean, tau_max = bed_shear_stresses(d_50, n, A_w_a, u_m_top, rho_w)

print(f'wave_friq_coef = {wave_friction_coefficient} \ntau_bed_wave = {tau_bed_wave:.2f} Pa\ntau_bed_current = {tau_bed_current:.2f} Pa\ntau_mean = {tau_mean:.2f} Pa\ntau_max = {tau_max:.2f} Pa')



wave_friq_coef = 0.1608952040952697 
tau_bed_wave = 51.16 Pa
tau_bed_current = 0.95 Pa
tau_mean = 2.03 Pa
tau_max = 53.19 Pa


In [152]:
def mobility_number(rho_stone=rho_stone, rho_w=rho_w, d_50=d_50, g=g, nu=nu, tau_max=tau_max):
    delta_s = (rho_stone - rho_w) / rho_w
    d_star = d_50 * (delta_s * g / nu**2)**(1/3)
    shields_crit = 0.30 / (1 + 1.2 * d_star) + 0.055 * (1 - np.exp(-0.02 * d_star))
    shields_combined = tau_max / (delta_s * g * d_50 * rho_w)
    MOB_top = shields_combined / shields_crit

    return d_star, shields_crit, shields_combined, MOB_top

d_star, shields_crit, shields_combined, MOB_top = mobility_number()
print(f'd_star = {d_star:.2f}\nshields_crit = {shields_crit:.4f}\nshields_combined = {shields_combined:.4f}\nMOB_top = {MOB_top:.4f}')

d_star = 5313.98
shields_crit = 0.0550
shields_combined = 0.0122
MOB_top = 0.2220
